# The Newsvendor Model - Section 3: Continuous Demand

In Sections 1 and 2 we explored the basic ideas of the Newsvendor Problem through numerical simulation and mathematical analysis.  By the end of Section 2, you had derived formulas for the expected profit in terms of the order quantity for any retail price $p$, wholesale cost $c$, and demand distribution $D \sim \text{Uniform}\{0,\dots,d\}$.

Here we will show that this analysis extends to continuous demand distributions.  $D \sim \text{Exponential}(\lambda)$.  We will derive everything analytically and use some Python at the end to help with visualizations.

The cumulative distribution function (cdf) and probability density function (pdf) of this distribution are:

$$ F(x) = \mathsf{P}(D \leq x) = 1 - e^{-\lambda x} \qquad f(x) = \frac{d F(x)}{dx} = \lambda e^{-\lambda x} $$

## Formulas

You may have already noticed that the formulas for sales, revenue, cost, and profit you came up with back in Section 1 Exercise 1.3 hold true for any demand random variable $D$ that has a non-negative probability distribution:

$$\begin{aligned}
    \text{sales} &= \min\{y, D\} \\
    \text{revenue} &= p \min\{y, D\} \\
    \text{cost} &= c y \\
    \text{profit} &= p \min\{y, D\} - c y
  \end{aligned} $$

However, the formulas for the *expected* sales, revenue, and profit, will change depending on the demand distribution (and therefore, so will the optimal order quantity that maximizes expected profit).

## Expected Sales

Recall that our basic recipe consisted of the following:

1. Find a formula for $\mathsf{E}[\text{profit}]$ as a function of order quantity $y$.
1. Maximize $\mathsf{E}[\text{profit}]$ with respect to order quantity $y$.


Finding a formula for expected profit as a function of order quantity is is really about finding a formula for expected sales, $\mathsf{E}[\text{sales}] = \mathsf{E}[\min\{y, D\}]$.  Since $\mathsf{E}[\text{profit}] = p \mathsf{E}[\text{sales}] - c y$.

### Exercise 3.1

Show that $\displaystyle\mathsf{E}[\text{sales}] = \frac{1}{\lambda} \left(1 - e^{-\lambda y}\right)$.

*Hint:* The hint for Exercise 2.1 can be translated to a continuous distribution:
$$\begin{aligned}
    \mathsf{E}[\text{sales}] = \mathsf{E}[\min\{y, D\}] &= \int_0^\infty \min\{y, x\} f(x)\ dx = \int_0^y x f(x)\ dx + \int_y^\infty y f(x)\ dx
  \end{aligned}$$
where $f(x)$ is the pdf of the distribution.  Remember to use [integration by parts](https://en.wikipedia.org/wiki/Integration_by_parts) to evaluate the first integral.

### Exercise 3.2

Show that the optimal order quantity (that maximizes expected profit) is given by the formula:

$$ y^* = \frac{1}{\lambda} \log\left(\frac{p}{c}\right) $$

### Exercise 3.3

Derive a formula for the optimal order quantity assuming $D \sim \text{Uniform}[0,d]$.

### Exercise 3.4

Derive a formula for the optimal order quantity for **any** demand distribution that is continuous and non-negative.  Denote its cdf as $F(x)$ and its pdf as $f(x)$.

To get a guess for the general formula, try rewriting the formulas you derived in Exercises 3.2 and 3.3 as $F(y^*) = \dots$

*Hints:* This simply combines [integration by parts](https://en.wikipedia.org/wiki/Integration_by_parts) with the [fundamental theorem of calculus](https://en.wikipedia.org/wiki/Fundamental_theorem_of_calculus).
$$\begin{aligned}
    \int_0^y x f(x) dx &= y F(y) - \int_0^y F(x) dx \\
    \int_y^\infty f(x) dx &= 1 - F(y) \\
    \frac{d}{dy} \left(\int_0^y F(x) dx\right) &= F(y)
  \end{aligned}$$

## Setup

The following Python code imports the necessary Python packages and specifies the problem parameters for the remainder of this notebook.  You can change the parameters if you'd like and all the code should still work, but their output may be mismatched with the written text.

In [1]:
# Import Python packages
import newsvendor
import news_plot

# Default exponential distribution has parameter lambda = 0.1
# Default model parameters are wholesale_cost = 1, retail_price = 2.
news = newsvendor.Newsvendor('exponential')

# Simulate demand
news.sim_data(num_days=365)

Loading BokehJS ...

In [2]:
bins = int(max(news.data['demand'])-min(news.data['demand'])+1)
fields = ['demand', 'sales', 'profit']
hist_func = lambda n: news_plot.hist_glyphs(news.data, bins=bins)
labels = {field: {'x': field, 'y': 'count', 'title': field + ' histogram'} for field in fields}
newsvendor.interact_gridplot(news_plot.figures(fields), hist_func, news, labels=labels)

interactive(children=(IntSlider(value=15, description='Order Qty', max=25), Output()), _dom_classes=('widget-i…

In [3]:
fields = ['sales', 'revenue', 'profit']
labels = {field: {'x': 'demand', 'y': field, 'title': field} for field in fields}
scatter_func = lambda n: news_plot.scatter_glyphs(n.data, 'demand')
newsvendor.interact_gridplot(news_plot.figures(fields), scatter_func, news, labels=labels)

interactive(children=(IntSlider(value=15, description='Order Qty', max=25), Output()), _dom_classes=('widget-i…

In [4]:
# Table of expected values with order quantity slider
fields = ['demand', 'sales', 'profit']
newsvendor.order_qty_interact(lambda y: news.exp_vals([y])[fields].style.hide_index(), news)

interactive(children=(IntSlider(value=15, description='Order Qty', max=25), Output()), _dom_classes=('widget-i…

In [5]:
fields = ['sales', 'revenue', 'cost', 'profit']
labels = {field: {'x': 'order qty', 'y': 'expected ' + field, 'title': field} for field in fields}
ev_func = lambda n: news_plot.line_glyphs(n.exp_vals(), 'order qty')
ev_sources, ev_plots = newsvendor.news_gridplot(news_plot.figures(fields, plot_width=350, plot_height=350), ev_func, news, labels=labels)
circle_func = lambda n: news_plot.circle_markers(n.exp_vals([n.order_qty]))
newsvendor.interact_gridplot({field: ev_plots[field] for field in fields}, circle_func, news, ncols=2)

interactive(children=(IntSlider(value=15, description='Order Qty', max=25), Output()), _dom_classes=('widget-i…